In [1]:
import requests
import re
import scrapy
import requests

from scrapy.http import TextResponse

In [2]:
!rm -rf musinsa

In [3]:
!scrapy startproject musinsa

New Scrapy project 'musinsa', using template directory 'c:\ds\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\DSS12\musinsa2\musinsa

You can start your first spider with:
    cd musinsa
    scrapy genspider example example.com


# 무신사 URL 확인

In [4]:
musinsa_url = "https://store.musinsa.com/app/brand/goods_list/musinsastandard?gclid=CjwKCAjw1v_0BRAkEiwALFkj5pkdoqVQmkznBLJ6ONQcJEV-UoAty7MYo5HuKCP2Xh0s3IPJAs_IORoCmygQAvD_BwE"
print(musinsa_url)

https://store.musinsa.com/app/brand/goods_list/musinsastandard?gclid=CjwKCAjw1v_0BRAkEiwALFkj5pkdoqVQmkznBLJ6ONQcJEV-UoAty7MYo5HuKCP2Xh0s3IPJAs_IORoCmygQAvD_BwE


### 페이지 파싱

In [5]:
range, page, = "1d", 1 

url = "https://store.musinsa.com/app/contents/bestranking/\
?d_cat_cd=&u_cat_cd=&range={}&price=&ex_soldout=&sale_goods=&new_product_yn=&list_kind=&page={}\
&display_cnt=90&sex=&popup=&sale_rate=&price1=&price2=&chk_new_product_yn=&chk_sale=&chk_soldout=".format(range, page)

req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

### 배너 있는 상품 링크

In [6]:
links_1 = response.xpath('//*[@id="searchList"]/li/div[3]/div[2]/p[2]/a/@href').extract()
links_1 = list(map(lambda data:response.urljoin(data), links_1))

### 배너 없는 상품 링크

In [7]:
links_2 = response.xpath('//*[@id="searchList"]/li/div[2]/div[2]/p[2]/a/@href').extract()
links_2 = list(map(lambda data:response.urljoin(data), links_2))

In [8]:
links = links_1 + links_2

In [9]:
len(links), links[0]

(90, 'https://store.musinsa.com/app/product/detail/1015777/0')

### 세부페이지 content 파싱

In [10]:
req2 = requests.get(links[0])
response = TextResponse(req2.url, body=req2.text, encoding="utf-8")

In [11]:
# 상품명
product_name = response.xpath('//*[@id="page_product_detail"]/div[4]/div[3]/span/span[1]/text()').extract()[0]

product_name

'[20SS] Cargo Jogger Slacks Pants - Black'

In [12]:
# 브랜드명
brand_name = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()').extract()[0]

brand_name

'NOT4NERD'

In [13]:
# 상품 품번
product_num = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/text()').extract()[1]

product_num

' PT021BK'

In [14]:
# 성별
gender = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span/span/text()').extract()[0]

gender

'남'

In [15]:
# 좋아요 갯수
good_num = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[5]/p[2]/span/text()').extract()[0]

good_num

'4,894'

In [16]:
# 구매후기 갯수
reviw_count = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[6]/p[2]/a/text()').extract()[0]

reviw_count

'1,339건'

In [17]:
# 판매가격
orgin_price = response.xpath('//*[@id="goods_price"]/del/text()').extract()[0]

orgin_price

'89,000'

In [18]:
# 할인가격
sale_price = response.xpath('//*[@id="sale_price"]/text()').extract()[0]

sale_price

' 49,000'

In [19]:
product_name, brand_name, product_num, gender, good_num, reviw_count, orgin_price, sale_price

('[20SS] Cargo Jogger Slacks Pants - Black',
 'NOT4NERD',
 ' PT021BK',
 '남',
 '4,894',
 '1,339건',
 '89,000',
 ' 49,000')

# Items.py 변환

In [27]:
%%writefile musinsa/musinsa/items.py
import scrapy


class MusinsaItem(scrapy.Item):
    product_name = scrapy.Field()
    brand_name = scrapy.Field()
    prproduct_num = scrapy.Field()
    gender = scrapy.Field()
    good_num = scrapy.Field()
    reviw_count = scrapy.Field()
    orgin_price = scrapy.Field()
    sale_price = scrapy.Field()

Overwriting musinsa/musinsa/items.py


# Spider 생성

In [ ]:
%%writefile musinsa/musinsa/spiders/spider.py

import scrapy
import requests

from musinsa.items import MusinsaItem
from scrapy.http import TextResponse

class MusinsaSpider(scrapy.Spider):
    
    name = "Musinsa"

    def __init__(self) :
        self.basic_url = "https://store.musinsa.com/app/contents/bestranking/?d_cat_cd=&u_cat_cd=&range=1d&price=&ex_soldout=&sale_goods=&new_product_yn=&list_kind=&page={}&display_cnt=90&sex=&popup=&sale_rate=&price1=&price2=&chk_new_product_yn=&chk_sale=&chk_soldout=".format(page)
        self.start_urls = []
        last_page_num = self.get_last_page
        for page in range(last_page_num+1):
            self.start_urls.append(self.base_url)
        
    def get_last_page(self):
        req = requests.get(self.base_url)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        last_page = response.xpath('//*[@id="product_list"]/div[2]/div[5]/div/div/a[14]').extract()
        last_page_num = re.split('\s', last_page[0])[3]
        last_page_num = re.findall('\d+', last_page_num)
        last_page_num = list(map(int, last_page_num))[-1]
        return last_page_num
    
    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse_link(self, response):
        links_1 = response.xpath('//*[@id="searchList"]/li/div[3]/div[2]/p[2]/a/@href').extract()
        links_1 = list(map(lambda data:response.urljoin(data), links_1))
        links_2 = response.xpath('//*[@id="searchList"]/li/div[2]/div[2]/p[2]/a/@href').extract()
        links_2 = list(map(lambda data:response.urljoin(data), links_2))
        links = links_1 + links_2
        for link in links:
            yield scrapy.Request(link, callback=self.parse_page_contents)
                    
    def parse_content(self, response):
        item = MusinsaItem()
        item["product_name"] = response.xpath('//*[@id="page_product_detail"]/div[4]/div[3]/span/span[1]/text()').extract()[0]
        item["brand_name"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()').extract()[0]
        item["product_num"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/text()').extract()[1].replace(" ","")
        item["gender"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span/span/text()').extract()
        item["good_num"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[5]/p[2]/span/text()').extract()
        item["reviw_count"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[6]/p[2]/a/text()').extract()
        item["orgin_price"] = response.xpath('//*[@id="goods_price"]/del/text()').extract()[0]
        item["sale_price"] = response.xpath('//*[@id="sale_price"]/text()').extract()[0].replace(" ","") 
        yield item

In [ ]:
%%writefile run.sh
cd musinsa
scrapy crawl Musinsa -o ranking.csv

In [ ]:
!ls -al run.sh

In [ ]:
import os
os.chmod("run.sh", 0o764)

In [ ]:
!ls -al run.sh

In [ ]:
!./run.sh